In [3]:
import re
import os
import shutil
import pandas as pd

# Borrar train folder

In [58]:
path = '/home/mr1142/Documents/Data/logs/fit'

In [59]:
for di in os.listdir(path):
    shutil.rmtree(os.path.join(path, di, 'train'), ignore_errors=True)

# Borrar modelos malos

In [55]:
path = '/home/mr1142/Documents/Data/models/validation_results/validation_results' + '' + '.csv'
df = pd.read_csv(path)

In [57]:
df.sort_values('dice_coef_loss').head(20)

,type,name,loss,dice_coef_loss,loss_mask,accuracy,AUC,false_positives,false_negatives
38,uloss,final_renacimiento_validation_8,0.100469,0.026820,0.073125,0.978087,0.979268,68079.0,75530.0
20,unet,final_renacimiento_validation_6,0.027842,0.027473,0.078634,0.977579,0.977706,72640.0,74298.0
37,unet,final_renacimiento_validation_7,0.028393,0.027809,0.077276,0.977255,0.977421,72967.0,76094.0
65,unet,unet_final_renacimiento_validation_6_fine_tuning,0.028354,0.027965,0.080699,0.977174,0.977285,74099.0,75491.0
19,uloss,final_renacimiento_validation_6,0.102747,0.028376,0.073854,0.976893,0.977792,70053.0,81382.0
68,unet,unet_final_renacimiento_validation_6_fine_tuning,0.029598,0.029172,0.081365,0.976225,0.975924,73761.0,82054.0
67,unet,unet_final_renacimiento_validation_6_fine_tuning,0.030311,0.029861,0.083249,0.975673,0.975210,75008.0,84418.0
76,unet,unet_final_renacimiento_validation_6_fine_tuni...,0.030328,0.029898,0.082915,0.975544,0.975894,85260.0,75012.0
71,uloss,uloss_final_renacimiento_validation_8_fine_tun...,0.111469,0.030014,0.080992,0.975462,0.977338,87655.0,73155.0
18,unet,final_renacimiento_validation_5,0.030735,0.030172,0.084180,0.975373,0.975297,76188.0,85209.0


In [40]:
malos_ = df[['type', 'name', 'dice_coef_loss']][df.dice_coef_loss>0.05]

In [41]:
malos_

,type,name,dice_coef_loss
78,unet,new_segmentation_6,0.423076


In [42]:
malos = [df['type'][i] + '_' + df['name'][i] for i in malos_.index]
malos = [re.sub('__', '_', mal) for mal in malos]
malos = [re.sub('.h5', '', mal) for mal in malos]
malos = [re.sub('patologic_', '', mal) for mal in malos]

In [43]:
malos

['unet_new_segmentation_6']

In [44]:
def match(lista_resultados, lista_match):
    return [f for f in lista_resultados if any([bool(re.search(match, f)) for match in lista_match])]

In [45]:
def eliminar(path, malos):
    files = os.listdir(path)
    files = match(files, malos)
    if len(files) == len(malos):
        for file in files:
            try:
                os.remove(os.path.join(path, file))
                print(f'file {file} removed')
            except:
                shutil.rmtree(os.path.join(path,file), ignore_errors=True)
                print(f'folder {file} removed')
    else:
        print('no tienen el mismo tamaño')
        print(files)

### Logs

In [46]:
path = '/home/mr1142/Documents/Data/logs/fit'

In [47]:
eliminar(path, malos)

folder unet_new_segmentation_6 removed


### Modelos

In [48]:
path = '/home/mr1142/Documents/Data/models'

In [49]:
eliminar(path, malos)

file unet_new_segmentation_6.h5 removed


# Borrar datos

In [52]:
path = '/home/mr1142/Documents/Data/models/validation_results'
csvs = os.listdir(path)

for csv in csvs:
    df = pd.read_csv(os.path.join(path, csv))
    index = [i for i in df.index if any([bool(re.search(mal, df['name'][i])) for mal in malos_['name']])]
    df = df.drop(index)
    df.to_csv(os.path.join(path, csv), index = False)

KeyError: 'name'